In [8]:
from lxml import etree

def rename_attr(e,a1,a2,*,maybe=False):
    if maybe and a1 not in e.attrib: return
    e.attrib[a2]=e.attrib[a1]
    del e.attrib[a1]

def rename_tags(top):
    for e in top.findall('.//verse'): e.tag='lg'
    for e in top.findall('.//line'): e.tag='l'
    for e in top.findall('.//tr'): e.tag='row'
    for e in top.findall('.//td'): e.tag='cell'
    for e in top.findall('.//table'): rename_attr(e,'layout_type','rend')
    for e in top.findall('.//span'): rename_attr(e,'family','rend',maybe=True)
    for e in top.findall('.//TODO'):
        e.tag='note'
        e.attrib['type']='TODO'
    for e in top.findall('.//enum'):
        e.tag='list'
        e.attrib['type']='numbered'
        rename_attr(e,'labels','subtype')
    for e in top.findall('.//li'): e.tag='item'
    for e in top.findall('.//printed_page'):
        rename_attr(e,'edition','ed')
        e.tag='pb'
        if 'page_id' in e.attrib: rename_attr(e,'page_id','pdf_page')
        e.attrib['n']=e.text
        e.text=None
    for e in top.findall('.//footnote'):
        e.tag='note'
        e.attrib['place']='foot'
        rename_attr(e,'mark','n')
    for e in top.findall('.//ref'):
        e.tag='ptr'
    for e in top.iter():
        if not e.tag.startswith('struct-'): continue
        e.attrib['type']=e.tag[len('struct-'):]
        e.tag='div'
        if 'hanging' in e.attrib:
            del e.attrib['hanging']
            e.attrib['rend']='hanging'
        rename_attr(e,'toc_num','id',maybe=True)
        # rename_attr(e,'sect_num','n',maybe=True)
    for e in top.findall('.//heading'):
        e.tag='head'
        rename_attr(e,'toc_num','n',maybe=True)
    for e in top.findall('.//p'):
        rename_attr(e,'anchor','id',maybe=True)
        rename_attr(e,'num','n',maybe=True)
        if 'editorial_title' in e.attrib:
            e.attrib['type']='editorial_title'
            del e.attrib['editorial_title']
    return top

def _E(tag,text=None,subs=[],**kw):
    ret=etree.Element(tag,**kw)
    ret.text=text
    for s in subs: ret.append(s)
    return ret
    
def restructure(book):
    # reassemble structure (front, main, back)
    book.tag='TEI'
    book.insert(0,_E('teiHeader'))
    book.insert(1,teiText:=_E('text'))
    for e in book[2:]:
        teiText.append(e)
    for div in book.findall('.//div[@type="1-part"]'):
        if (name:=div.attrib['name'])=='(Front)':
            div.tag='front'
            del div.attrib['name']
            del div.attrib['type']
            assert div[0].tag=='head'
            div.remove(div[0])
        elif name=='Part I':
            assert div.getparent()==teiText
            teiText.insert(teiText.index(div),teiMain:=_E('main'))
            teiMain.append(div)
        else: teiMain.append(div)
    return book

book=restructure(rename_tags(etree.parse('src/book.6.xml',etree.XMLParser(remove_blank_text=True)).getroot()))
open('src/book.6.tei','w').write(etree.tostring(book,encoding='unicode',pretty_print=True))

for fin in ['src/bib-1','src/index','src/gloss']:
    bib=rename_tags(etree.parse(fin+'.xml',etree.XMLParser(remove_blank_text=True)).getroot())
    open(fin+'.tei','w').write(etree.tostring(bib,encoding='unicode',pretty_print=True))
    